In [ ]:
# This program is for getting an understanding of how to use nested sampling

In [1]:
#=
TO DO LIST (calculating Kinetic Energy):
- make function for computing particle velocities (via verlet method)
        - is it possible to find velocity separate from finding new position...?
        - consider using velocity verlet instead (wikipedia page) of position verlet
- give velocities at end of MD simulation period
- use velocities to compute Kinetic Energy



TO DO LIST (nested sampling):
X Create initial random samples for nested sampling, define qualities
X compare samples' energies
X delete highest total energy sample and clone a different sample 
X walk the cloned sample, decorrelate by changing speeds
X walk all samples and then repeat energy check, clone, etc. ...
X do this for some iterations, then compare original max sample energy to the final max sample energy

TO DO LIST (heat capacity):
- Record highest potential energies
- Make sure each potential is lower than the last
- Calculate partition function
- Cv = -(δ/δT δlnZ/δβ)v
    - β = 1/(kT)
    - k = kB = Bolzmann constant
    - En = energy recorded at level n
    UNITLESS
    - K = num_samples
    - wn = α^n - α^(n+1)
    - α = (K-1)/K
    - N = number of particles in a sample
    - Zest  = Σn [α^n - α^(n+1)] * [exp(-β*E(xj))]
        (this only works for our specific choice of α)
- GOAL: T in kelvins, Cv in eV per kelvin
* Try setting initital velocities to 0, or have them far apart
X Make sure potential energy drops to zero at σ

TO DO LIST (phase diagram):
- Figure out how to choose set of pressures
- Learn how to use NS results to calculate heat capacity
- Learn how to find maxima on p-T curves
- Graph results
=#

"""Go through function definitions and remove all global variables"""

"Go through function definitions and remove all global variables"

In [2]:
#=
Nested Sampling Algorithm STEPS:
1. Generate K random samples (walkers) uniformly in the total phase space volume (Potential Energy Surface)
2. Calculate energies of each sample
3. Delete highest energy sample with energy E1
4. Generate new sample with E<E1
    - Clone a random sample and perform a random walk until it is independent from the parent configuration.
        - Markov Chain Monte Carlo
        - Total Enthalpy Hamiltonian Monte Carlo
        - Galilean Monte Carlo
5. Repeat many times...
6. Reach bottom, calculate thermodynamic quantities.

• Do samples of 64 LJ atoms at constant pressure

p-T Phase Diagram STEPS:
1. Choose a set of pressures and do NS calculations for each pressure
2. Calculate cp(T) for every sampled pressure.
3. Locate maxima on cp(T) curves.
=#

In [1]:
# Run this cell ONCE; it takes time and doesn't need to be run more than once
# Rerun if you shutdown the kernel
using Pkg
Pkg.add("Plots")
using Plots
gr(title = "no title", legend = :none, background_color = :black)

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.1/Manifest.toml`
 [no changes]


Plots.GRBackend()

In [2]:
#=
Calculating force of particle j on particle i
=#


function comp_force(sqς, xi, xj, yi, yj, Fx, Fy, i, r = (xi - xj)^2 + (yi - yj)^2)
    
    # Check if i and j are too far apart to matter
    if r < sqς
        r = sqrt(r)
        
        # Calculate magnitude of force from particle j
        F = (-24*r^(-7) + 48*r^(-13)) - F_cutoff
        
        #F_cutoff is the tiny bit of force at r = rsqς
        
        # Calculate x and y components of force from j, then add to net force on i
        Fx[i] += (xi - xj) / r * F  # cos(theta) * F
        Fy[i] += (yi - yj) / r * F  # sin(theta) * F 
    end
    
end

comp_force (generic function with 2 methods)

In [3]:
#=
Using Verlet method to get new position
=#

function newposition(K,L,z,zprev,Fz,dt,vz,n,T)
    
    for i in 1:K
    
        znew = 2z[i] - zprev[i] + Fz[i]*dt^2
        
        
        # At final timestep, calculate velocities
        #if n == T
        vz[i] = (znew - zprev[i]) / (2dt)
        #end
        #=
        Running velocity calculation here is not ideal 
        since this logic check gets run at every loop of particle path calculation.
        However, the velocity calculation needs the old z and zprev, 
        not the new one given at the end of this function, 
        so it needs to be calculated inside this function.
        =#
        
        
        # If new position falls out of bounds, pull it in on opposite side
        # Also do this for previous position so it doesn't muck up later calculations 
        if znew >= L
            zprev[i] = z[i] - 2L
            z[i] = znew - 2L
        elseif znew < -L
            zprev[i] = z[i] + 2L
            z[i] = znew + 2L
        else
            zprev[i] = z[i]
            z[i] = znew
        end
        
        
    end
        
end

newposition (generic function with 1 method)

In [4]:
#=
Takes each particle's current coordinates and saves them to a small array, 
then saves that small array to a bigger array. It will be used to animate our particles.
Each small array has the information to plot one frame of an animation.
=#

function recordpositions(K,x,y,x_for_gif,y_for_gif)
    
    # Initialize for adding array to plot in gif
    temp_x = []
    temp_y = []   
    
    for i in 1:K
        # Create small list of particle positions for one frame
        push!(temp_x, x[i])
        push!(temp_y, y[i]) 
    end

    # Push small lists to large arrays
    push!(x_for_gif, temp_x)
    push!(y_for_gif, temp_y)
    
end

recordpositions (generic function with 1 method)

In [5]:
#=
Takes each particle's current velocities and saves them to a small array, 
then saves that small array to a bigger array. It's not used for the animation.
Each small array has information about one frame of the animation.
=#

function recordvelocities(K,vx,vy,vx_tmp,vy_tmp)
    
    # Initialize small arrays
    temp_vx = []
    temp_vy = []   
    
    for i in 1:K
        # Create small list of particle velocities of one frame
        push!(temp_vx, vx[i])
        push!(temp_vy, vy[i]) 
    end

    # Push small lists to large arrays
    push!(vx_tmp, temp_vx)
    push!(vy_tmp, temp_vy)
    
end

recordvelocities (generic function with 1 method)

In [6]:
#=
This function creates the nearest periodic version from the getgo by examining which boundaries each
particle is nearest to.
=#

function oneimage(L,ς,sqς,xi,xj,yi,yj,Fx,Fy,i)
    
    #= 
    Find minimum between distance to upper bound, lower bound, and cutoff distance. 
    Use result for creating  nearest periodic version of particle j. Repeat comparison with other coordinates.
    =#
    
    #= 
    In either dimension, there are 3 areas: near lower periodic boundary, near upper one, or near neither of them.
    
    If i coord is much higher than j coord, j will be transformed to be 2L higher.
    If i coord is much lower than j coord, j will be transformed to be 2L lower.
    If i coord and j coord are in about the same area, j coord will not be transformed.
    =#
    
    # Check if xi is within cutoff of boundary
    if (L - abs(xi)) >= ς
        nbxi = 0
    else
        nbxi = sign(xi)
    end
    
    
    # Check if xj is within cutoff of boundary
    if (L - abs(xj)) >= ς
        nbxj = 0
    else
        nbxj = sign(xj)
    end
    

    # Check if yi is within cutoff of boundary
    if (L - abs(yi)) >= ς
        nbyi = 0
    else
        nbyi = sign(yi)
    end
    
    
    # Find nearest boundary to yj
    # Check if yj is within cutoff of boundary
    if (L - abs(yj)) >= ς
        nbyj = 0
    else
        nbyj = sign(yj)
    end

    
    npx = xj + (abs(nbxi*nbxj))*sign(nbxi-nbxj)*2*L
    npy = yj + (abs(nbyi*nbyj))*sign(nbyi-nbyj)*2*L
    
    
    comp_force(sqς,xi,npx,yi,npy,Fx,Fy,i)
    
end

oneimage (generic function with 1 method)

In [7]:
#=
Takes the computation function you wish to use as a parameter, computes particle paths
=#

function comp_path(K,L,ς,sqς,x,y,Fx,Fy,chosen_way::Function)
    
    # Calculating net force on each particle one at a time
    
    
    for i in 1:K
        # Initialize forces as zero, we will add on forces from each other particle
        Fx[i] = 0
        Fy[i] = 0
        
        #= 
        If particle i is far from any system boundaries, do force calculations only inside main system.
        =#
        if L-abs(x[i])>=ς && L-abs(y[i])>=ς
                
            for j in 1:K

                # Check if i and j are same particle; particle i can't act on itself
                if i == j
                    continue
        
                else
                    comp_force(sqς,x[i],x[j],y[i],y[j],Fx,Fy,i)
                    
                end
                
            end
         
            
        #= 
        Since particle i is near a boundary, check if particle j 
        is also near any system boundaries. If so do force calculations
        with periodic versions of j. If not, use versions of j within system.
        =#
        else
            
            # Calculate forces from each other particle
            for j in 1:K

                # Check if i and j are same particle; particle i can't act on itself
                if i == j
                    continue
                    
                # If particle j is far from any system boundaries, do force calculations only inside main system.    
                elseif L-abs(x[j])>=ς && L-abs(y[j])>=ς
                    comp_force(sqς,x[i],x[j],y[i],y[j],Fx,Fy,i)
    
                else
                    chosen_way(L,ς,sqς,x[i],x[j],y[i],y[j],Fx,Fy,i) 
                    
                end # if i == j ...
                
            end # for j in 1:K
            
            
        end # if abs(L-x[i])>=3 && abs(L-y[i])>=3 ...
        
    end # for i in 1:K
    
end

comp_path (generic function with 1 method)

In [8]:
function comp_potential(sqς, xi, xj, yi, yj, LJ_PEs, i, r = (xi - xj)^2 + (yi - yj)^2)
    
    # Check if i and j are too far apart to matter
    if r < sqς
        r = sqrt(r)
        # Add UNITS
        # Calculate LJ potential from particle j
        LJ_PEs[i] += (4*ϵ*((σ/r)^12 - (σ/r)^6) - PE_cutoff)
        
        #PE_cutoff is the bit of potential at r = sqς
        
    end
    
end

comp_potential (generic function with 2 methods)

In [9]:
#=
This function creates the nearest periodic version from the getgo by 
examining which boundaries each particle is nearest to.
=#

function periodic_potential(L,ς,sqς,xi,xj,yi,yj,LJ_PEs,i)
    
    #= 
    Find minimum between distance to upper bound, lower bound, and cutoff distance. 
    Use result for creating  nearest periodic version of particle j. Repeat comparison with other coordinates.
    =#
    
    #= 
    In either dimension, there are 3 areas: near lower periodic boundary, near upper one, or near neither of them.
    
    If i coord is much higher than j coord, j will be transformed to be 2L higher.
    If i coord is much lower than j coord, j will be transformed to be 2L lower.
    If i coord and j coord are in about the same area, j coord will not be transformed.
    =#
    
    # Check if xi is within cutoff of boundary
    if (L - abs(xi)) >= ς
        nbxi = 0
    else
        nbxi = sign(xi)
    end
    
    
    # Check if xj is within cutoff of boundary
    if (L - abs(xj)) >= ς
        nbxj = 0
    else
        nbxj = sign(xj)
    end
    

    # Check if yi is within cutoff of boundary
    if (L - abs(yi)) >= ς
        nbyi = 0
    else
        nbyi = sign(yi)
    end
    
    
    # Find nearest boundary to yj
    # Check if yj is within cutoff of boundary
    if (L - abs(yj)) >= ς
        nbyj = 0
    else
        nbyj = sign(yj)
    end

    
    npx = xj + (abs(nbxi*nbxj))*sign(nbxi-nbxj)*2*L
    npy = yj + (abs(nbyi*nbyj))*sign(nbyi-nbyj)*2*L
    
    
    comp_potential(sqς,xi,npx,yi,npy,LJ_PEs,i)
    
end

periodic_potential (generic function with 1 method)

In [10]:
"""COSTANTS"""

# These values are in reduced units, defined by the properties of the element argon
ϵ = 1             # ENERGY, Depth of potential well, ϵ = kB * 120 K for Argon
m = 1             # MASS of an argon atom, "Reduced mass", determines other properties like ϵ, σ
σ = 1             # DISTANCE unit, 3.4Å for argon


N  = 4                    # number of particles on one side
K  = N^2                  # total number of particles
num_particles = K
ς = 3 * σ                 # Max distance where interference is possible
sqς = ς^2                 # Square of max distance where interference is possible, useful for our functions
dt = 0.001 * sqrt(m*σ^2/ϵ) # time step size
ds = 1.3 * σ              # grid spacing
L  = 1.5 * N * ds         # system size
dr = 0.01 * σ             # max deviation from grid position
v0 = 0.3 * sqrt(ϵ/m)      # max initial velocity magnitude
vxavg = 0 * v0            # system average velocity in x direction
vyavg = 0 * v0            # system average velocity in y direction

F_cutoff = (-24*ς^(-7) + 48*ς^(-13)) # Force at cutoff distance. Subtract from force so F at ς is zero.
PE_cutoff = 4*ϵ*((σ/ς)^12 - (σ/ς)^6) # Potential at cutoff distance. Subtract from pot. so PE at ς is zero.


T = 2000 # number of timesteps to take
s = 10 # Desired number of iterations between data recorded for plots

if T%s != 0
    println("Number of timesteps (T) must be evenly divisible by (s)!")
    println("Change (s) or (T) so that [T%s == 0]")
end


#kB = 1.380649*10^(-23)  #Boltzmann constant in units of J/K
kB = 8.617333*10^(-5)  #Boltzmann constant in units of eV/K

# UNIT CONVERSIONS
# These will be used to convert the calculated energies into useful units.
# The following values are true for Argon.
ϵ_val = kB * 120 / ϵ              # energy
m_val = 6.6335209*10^(-26) / m    # kilograms
σ_val = 3.4*10^(-10) / σ          # meters
t_val = sqrt(m_val*σ_val^2/ϵ_val) # seconds, should be 1.8x10^-12...
v_val = sqrt(ϵ_val/m_val)         # m/s

3.948253326911903e11

In [11]:
#=
Takes the computation function you wish to use as a parameter, computes particle paths
=#

function get_energy(K,L,m,ς,sqς,x,y,vx,vy)
    
    # Calculating potential energy and kinetic energy of each particle
    
    # Initialize potential and kinetic energy vectors for summing later
    LJ_PEs = zeros(K)
    #LJ_KEs = zeros(K)
    
    for i in 1:K
        
        #= 
        If particle i is far from any system boundaries, do potential calculations only inside main system.
        =#
        if L-abs(x[i])>=ς && L-abs(y[i])>=ς
                
            for j in 1:K

                # Check if i and j are same particle; particle i can't act on itself
                if i == j
                    continue
        
                   
                else
                    comp_potential(sqς,x[i],x[j],y[i],y[j],LJ_PEs,i)
                    
                end
                
            end
         
            
        #= 
        Since particle i is near a boundary, check if particle j 
        is also near any system boundaries. If so, do potential calculations
        with periodic versions of j. If not, use versions of j within system.
        =#
        else
            
            # Calculate potentials from each other particle
            for j in 1:K

                # Check if i and j are same particle; particle i can't act on itself
                if i == j
                    continue
                    
                # If j is far from any system boundaries, do potential calculations only inside main system.    
                elseif L-abs(x[j])>=ς && L-abs(y[j])>=ς
                    comp_potential(sqς,x[i],x[j],y[i],y[j],LJ_PEs,i)
    
                else
                    periodic_potential(L,ς,sqς,x[i],x[j],y[i],y[j],LJ_PEs,i) 
                    
                end # if i == j ...
                
            end # for j in 1:K
            
            
        end # if abs(L-x[i])>=3 && abs(L-y[i])>=3 ...
        
        
        #LJ_KEs[i] = m/2*(vx[i]^2+vy[i]^2)
                
    end # for i in 1:K
    
    PE_total = sum(LJ_PEs) * ϵ_val     # ϵ_val makes it in units of energy I think...
    # Heat capacity calculation doesn't use kinetic energy...
    # KE_total should be adding to PE_total to the same E total value always...
    # KE_total = sum(LJ_KEs) * KE_factor #* m_val * v_val^2
    # E_total = PE_total + KE_total
    
    return PE_total
    
    end # Function

get_energy (generic function with 1 method)

In [12]:
### INITIAL SAMPLES

#= 
Work could be done to make samples have differing average velocities
    or other things that could help differentiate them.
=#

function make_sample(index)
    #= 
    Each sample consists of an array of the current x, y, vx, vy, 
        xprev, and yprev values.
    Each one also has an index, given while the sample is created.
    The index may or may not be useful later, but it's easy to add.
    =#
    
    # Initialize vectors
    init_x = []
    init_y = []
    init_vx = []
    init_vy = []
    init_xprev = []
    init_yprev = []

    k = 1 #iterator for creating particle grid


    # Create grid of NxN particles
    for i in 1:N
        for j in 1:N
            # Create coordinates for particle ij, move so grid center is at origin, 
            # spaced as far as 2*L/N apart, or as low as 0.5*σ apart, then displace randomly
            push!(init_x,((i-1) - (N-1)/2)*L/N + (rand() - 0.5)*dr)
            push!(init_y,((j-1) - (N-1)/2)*L/N + (rand() - 0.5)*dr)

            # Randomize initial velocity for calculating ghost previous position
            push!(init_vx,(rand() - 0.5)v0 + vxavg)
            push!(init_vy,(rand() - 0.5)v0 + vyavg)

            # Use initial velocity for initializing ghost previous position
            push!(init_xprev,init_x[k] - init_vx[k]*dt)
            push!(init_yprev,init_y[k] - init_vy[k]*dt)

            k += 1
        end
    end

    # This is the sample that will be returned.
    init_vals = [init_x, init_y, init_vx, init_vy, init_xprev, init_yprev, index]
    
    
    return init_vals
    
end

make_sample (generic function with 1 method)

In [13]:
### Calculate particle paths using each method and record points for plotting

function walk_record(sample,x_for_anim_i,y_for_anim_i)


    x = deepcopy(sample[1])
    y = deepcopy(sample[2])
    vx = deepcopy(sample[3])
    vy = deepcopy(sample[4])
    xprev = deepcopy(sample[5])
    yprev = deepcopy(sample[6])
    
    
    x_anim_tmp = []
    y_anim_tmp = []
    
    
    # Initialize force vectors and arrays for plotting
    Fx = zeros(K)
    Fy = zeros(K)

    
    for n in 1:T

        # Calculating net force on each particle one at a time
        comp_path(K,L,ς,sqς,x,y,Fx,Fy,oneimage)

        # Updating coordinates and velocities Verlet method
        newposition(K,L,x,xprev,Fx,dt,vx,n,T)
        newposition(K,L,y,yprev,Fy,dt,vy,n,T)

        
        # Record the points every few time steps
        if n%s == 0
            recordpositions(K,x,y,x_anim_tmp,y_anim_tmp)
            
            # Final timestep, record the velocities and new initial values
            if n == T
            #recordvelocities(K,vx,vy,vx_tmp,vy_tmp)
                
            sample[1] = deepcopy(x)
            sample[2] = deepcopy(y)
            sample[3] = deepcopy(vx)
            sample[4] = deepcopy(vy)
            sample[5] = deepcopy(xprev)
            sample[6] = deepcopy(yprev)    
                
            end
            
        end

        
    end

    
    # Use velocities/positions to calculate kinetic/potential 
    #energies and find total energy
    # Only potential energy is needed for heat capacityy calc...
    PE_total = get_energy(K,L,m,ς,sqς,x,y,vx,vy)
    
    
    print("Total potential energy: ",PE_total,"\n")   
      
    
    # Push each recorded value into the array for holding coordinates
    for i in 1:length(x_anim_tmp)
        push!(x_for_anim_i, deepcopy(x_anim_tmp[i]))
        push!(y_for_anim_i, deepcopy(y_anim_tmp[i]))
    end
    
    #=
    println("Sample ", sample[7], " has finished walking.\n\n")
    =#
    
    #return x_for_anim, y_for_anim
    
end

walk_record (generic function with 1 method)

In [14]:
### Calculate particle paths using each method and record points for plotting

function walk_energies(sample,PE_list,timesteps)


    x = deepcopy(sample[1])
    y = deepcopy(sample[2])
    vx = deepcopy(sample[3])
    vy = deepcopy(sample[4])
    xprev = deepcopy(sample[5])
    yprev = deepcopy(sample[6])

    
    # Initialize force vectors and arrays for plotting
    Fx = zeros(K)
    Fy = zeros(K)

    
    for n in 1:T

        # Calculating net force on each particle one at a time
        comp_path(K,L,ς,sqς,x,y,Fx,Fy,oneimage)

        # Updating coordinates and velocities Verlet method
        newposition(K,L,x,xprev,Fx,dt,vx,n,T)
        newposition(K,L,y,yprev,Fy,dt,vy,n,T)

        
        # Record the points every few time steps
        if n%s == 0
            # Use velocities/positions to calculate kinetic/potential energies and find total energy
            # Only potential energy is needed....
            PE_total = get_energy(K,L,m,ς,sqς,x,y,vx,vy)
            
            push!(PE_list, PE_total)
            push!(timesteps, n)
            
            # Final timestep, record the velocities and new initial values
            if n == T
            #recordvelocities(K,vx,vy,vx_tmp,vy_tmp)
                
            sample[1] = deepcopy(x)
            sample[2] = deepcopy(y)
            sample[3] = deepcopy(vx)
            sample[4] = deepcopy(vy)
            sample[5] = deepcopy(xprev)
            sample[6] = deepcopy(yprev)    
                
            end
            
        end

        
    end

    
      
    
    # only potential energy is needed
    return PE_list, timesteps
    
end

walk_energies (generic function with 1 method)

In [15]:
# Finding and replacing the sample with the maximum energy
"""Make sure the index matches the index assigned to the sample"""
function replace_max_sample(K,L,m,ς,sqς,samples_list,sample_energies)
    
    # Initialize array of energy totals
    #E_tot_list = []
    # only potential energy is needed
    PE_tot_list = []
      
    
    for i in 1:num_samples
        # Add each sample's total energy to the list
        x = deepcopy(samples_list[i][1])
        y = deepcopy(samples_list[i][2])
        vx = deepcopy(samples_list[i][3])
        vy = deepcopy(samples_list[i][4])
        PE_total = get_energy(K,L,m,ς,sqς,x,y,vx,vy)
        #push!(E_tot_list, E_total)
        push!(PE_tot_list, PE_total)
    end

    max_E, max_index = findmax(PE_tot_list)
    print("Sample ", max_index, " has the maximum energy, with PE_tot = ", max_E,"\n")
    
    # Record max energy for making heat capacity plot
    push!(sample_energies, deepcopy(max_E))
    
    # Pick a random index to use to create a new, lower energy sample
    replacement_index = rand(1:num_samples)
    # If the index to deepcopy matches the current one, pick a different one
    while replacement_index == max_index
        replacement_index = rand(1:num_samples)
    end
    
    # Copy the information from the random index over the max sample
    samples_list[max_index][1] = deepcopy(samples_list[replacement_index][1]) #x
    samples_list[max_index][2] = deepcopy(samples_list[replacement_index][2]) #y
    samples_list[max_index][3] = deepcopy(samples_list[replacement_index][3]) #vx
    samples_list[max_index][4] = deepcopy(samples_list[replacement_index][4]) #vy
    samples_list[max_index][5] = deepcopy(samples_list[replacement_index][5]) #xprev
    samples_list[max_index][6] = deepcopy(samples_list[replacement_index][6]) #yprev
    
    
    x = deepcopy(samples_list[max_index][1])
    y = deepcopy(samples_list[max_index][2])
    vx = deepcopy(samples_list[max_index][3])
    vy = deepcopy(samples_list[max_index][4])
    PE_total = get_energy(K,L,m,ς,sqς,x,y,vx,vy)
    
    
    print("Sample ",replacement_index," has replaced sample ",max_index,".\n")
    print("Sample ",max_index," now has a total energy of ",PE_total,".\n\n")
    
    
    
    # Use max_index to walk the replaced sample.
    return max_index
end

replace_max_sample

In [16]:
### This function walks a sample and slightly reduces its kinetic energy


function decorrelate(sample)


    OG_sample = deepcopy(sample)
    
    x = deepcopy(sample[1])
    y = deepcopy(sample[2])
    vx = deepcopy(sample[3])
    vy = deepcopy(sample[4])
    xprev = deepcopy(sample[5])
    yprev = deepcopy(sample[6])
    
    PE_total = get_energy(K,L,m,ς,sqς,x,y,vx,vy)
    
    new_PE_total = PE_total
    
    J = 0  # Variable to iterate, make sure loop doesn't get out of hand
    
    # Decrease magnitude of velocities slightly and randomly
    # at least .95 and not more than 1
    low_end = 0.95
    high_end = 1.00 - low_end
    
    
    while new_PE_total >= PE_total
        
        J += 1
        if J > 20
            print("Failed to decrease total energy within 20 iterations.\n")
            break
        end
        
        
        # Scale down velocities a little bit, change "prev" positions so algorithm works properly
        x = deepcopy(sample[1])
        y = deepcopy(sample[2])
        vx = (low_end+rand()*high_end) * deepcopy(sample[3])
        vy = (low_end+rand()*high_end) * deepcopy(sample[4])
        xprev = [];
        yprev = [];
        
        
        """I ripped this from the make_sample function, so it should work the same way..."""
        k = 1
        for i in 1:N
            for j in 1:N
                # Use initial velocity for reinitializing a ghost "previous position"
                push!(xprev,x[k] - vx[k]*dt)
                push!(yprev,y[k] - vy[k]*dt)
                k += 1
            end
        end
        
        # Initialize force vectors and arrays for plotting
        Fx = zeros(K)
        Fy = zeros(K)


        for n in 1:T

            # Calculating net force on each particle one at a time
            comp_path(K,L,ς,sqς,x,y,Fx,Fy,oneimage)

            # Updating coordinates and velocities Verlet method
            newposition(K,L,x,xprev,Fx,dt,vx,n,T)
            newposition(K,L,y,yprev,Fy,dt,vy,n,T)

            # Final timestep: check that total energy is less than intially; set new initial values if so
            if n == T
                
                new_PE_total = get_energy(K,L,m,ς,sqς,x,y,vx,vy)
                
                if new_PE_total < PE_total
                    # New energy total is less than originally, so update sample
                    sample[1] = deepcopy(x)
                    sample[2] = deepcopy(y)
                    sample[3] = deepcopy(vx)
                    sample[4] = deepcopy(vy)
                    sample[5] = deepcopy(xprev)
                    sample[6] = deepcopy(yprev)
                    print(new_PE_total," is LESS than the original ",PE_total,".\n")
                    break
                else
                    # Failed to decrease total energy, so reset to original values and try again
                    print(new_PE_total," is MORE than the original ",PE_total,".\n")
                end

            end

        end

    end   #While
    
    #=
    # Use velocities/positions to calculate kinetic/potential energies and find total energy
    new_E_total, new_PE_total, new_KE_total = get_energy(K,L,m,ς,sqς,x,y,vx,vy)
    
    print("Total potential energy changed from ",PE_total," to ",new_PE_total,".\n")
    print("Total kinetic energy changed from ",KE_total," to ",new_KE_total,".\n")
    print("Total energy decreased from ",E_total," to ",new_E_total,".\n")    
    =#
    
    println("Sample ", sample[7], " has finished decorrelating.\n\n\n")
    
end

decorrelate (generic function with 1 method)

In [17]:
# SAMPLE CREATION
# Initialize list of samples, matching plot names, and matching mp4 names

samples_list = [];
plotnames = [];
animfile_names = [];

num_samples = 6

for i in 1:num_samples
    sample_num = string(i)
    push!(samples_list, make_sample(i))
    push!(plotnames, "Sample "*sample_num)
    
    #= 
    If you mess with this line, be sure to go delete the old sample
    animations in tmp_animations!
    =#
    push!(animfile_names, "NestedSample_0"*sample_num)
end


In [18]:
"""RUNNING THIS CELL KILLS THE KERNEL"""
# TRACKING ENERGY TOTALS
"""TO DO: Make sure samples obey conservation of energy. Energy totals shouldn't change with time."""
#=
KE_factor = 1

PE_list = []
timesteps = []

#=
for j in 1:length(samples_list)
    walk_energies(samples_list[j],PE_list,timesteps)
end
=#

walk_energies(samples_list[1],PE_list,timesteps)

PEs    = plot(timesteps, PE_list, title = "Energy Over Time",legend = :best,label="PE",
         dpi = 150)
#plot!(PEs,timesteps, KE_list, label="KE")
#plot!(PEs,timesteps, tot_E_list, label="tot_E")

LoadError: syntax: incomplete: unterminated multi-line comment #= ... =#

In [19]:
# WALKING AND RECORDING

# Here the for_anim arrays are initialized as lists of empty arrays, one array per sample
x_for_anim = fill([],length(samples_list))
y_for_anim = fill([],length(samples_list))

# This loop makes the empty arrays independent from the each other in the for_anim arrays.
# VERY IMPORTANT
for i in 1:length(samples_list)
    x_for_anim[i] = []
    y_for_anim[i] = []
end

num_loops = 10

sample_energies = []


for i in 1:num_loops
    
    
    for j in 1:length(samples_list)
        #x_for_anim, y_for_anim = 
        walk_record(samples_list[j],x_for_anim[j],y_for_anim[j])
    end
    

    # CLONE A SAMPLE OVER MAX-ENERGY SAMPLE    
    max_index = replace_max_sample(K,L,m,ς,sqς,samples_list,sample_energies)
    # max energies recorded in sample_energies


    # WALK AND DECORRELATE CLONED SAMPLE
    decorrelate(samples_list[max_index]);
    
    
end

Total potential energy: -0.10950903719789286
Total potential energy: -0.15762376786361312
Total potential energy: -0.11970001622439085
Total potential energy: -0.1782286984156045
Total potential energy: -0.2089497208377944
Total potential energy: -0.12304941439679071
Sample 1 has the maximum energy, with PE_tot = -0.10950903719789286
Sample 2 has replaced sample 1.
Sample 1 now has a total energy of -0.15762376786361312.

-0.2087520007423909 is LESS than the original -0.15762376786361312.
Sample 1 has finished decorrelating.



Total potential energy: -0.19867244328878708
Total potential energy: -0.18890088896384058
Total potential energy: -0.17386389587317158
Total potential energy: -0.19781675956765654
Total potential energy: -0.18625189848053778
Total potential energy: -0.23390425391639766
Sample 3 has the maximum energy, with PE_tot = -0.17386389587317158
Sample 2 has replaced sample 3.
Sample 3 now has a total energy of -0.18890088896384058.

-0.22033351159150347 is LESS than the 

-0.25996242054219193 is LESS than the original -0.22054086228345404.
Sample 5 has finished decorrelating.



Total potential energy: -0.2502654738243256
Total potential energy: -0.19016876292437332
Total potential energy: -0.2502654738243256
Total potential energy: -0.2502654738243256
Total potential energy: -0.22678473367210225
Total potential energy: -0.16024012506705984
Sample 6 has the maximum energy, with PE_tot = -0.16024012506705984
Sample 4 has replaced sample 6.
Sample 6 now has a total energy of -0.2502654738243256.

-0.19279773793342203 is MORE than the original -0.2502654738243256.
-0.18320931868863308 is MORE than the original -0.2502654738243256.
-0.19471617456060172 is MORE than the original -0.2502654738243256.
-0.16479760250596287 is MORE than the original -0.2502654738243256.
-0.1907094212399174 is MORE than the original -0.2502654738243256.
-0.1904574388881582 is MORE than the original -0.2502654738243256.
-0.18142328645147704 is MORE than the original -0.25026547382

In [20]:
# Try out temperatures from 0 to 400 in increments of 10
maxT = 400
inc = 5
α = (num_samples - 1)/num_samples

"""TO DO: Figure out units!"""
Cv = zeros(convert(Int64,floor(maxT/inc)))
T = zeros(convert(Int64,floor(maxT/inc)))

for i in 1:length(Cv)
    T[i] = inc * i
    β = 1/(kB * T[i])
    
    
    # Calculate Zest
    Zest = 0   # Initialize value for summation
    for n in 1:length(sample_energies)
        Exj = sample_energies[1]
        # Exj = sample_energies[n]
        # """Figure out meaning of E(xj)"""
        wn = α^n - α^(n+1)      # Sample weight
        Zest += wn * exp(-β*Exj)
    end
    
    # Calculate Cv
    # Initialize values for summations
    sumA = 0
    sumB = 0
    sumC = 0
    for n in 1:length(sample_energies)
        En = sample_energies[n]
        wn = α^n - α^(n+1)      # Sample weight
        
        sumA += wn * En * exp(-β*En) / (kB*T[i]^2) / Zest^2
        sumB += wn * En * exp(-β*En)
        sumC += En^2 * exp(-β*En) / (kB*T[i]^2)
    end
    
    term1 = 3*num_particles/2 * kB
    term2 = sumA * sumB
    term3 = 1/Zest * sumC
    
    Cv[i] = term1 - term2 + term3
    print("temp: ",inc*i,"   Cv: ",Cv[i],"\n")
    print("Zest: ",Zest,"\n")
    print("term1: ",term1,"\n")
    print("sumA: ",sumA,"\n")
    print("sumB: ",sumB,"\n")
    print("sumC: ",sumC,"\n\n")
    
end

for i in 1:length(Cv)
    print("temp: ",inc*i,"   Cv: ",Cv[i],"\n")
end


temp: 5   Cv: -2.5580392652111605e129
Zest: 1.6771566963072913e110
term1: 0.0020681599200000006
sumA: -6.497159381098861e-45
sumB: -3.9371656368056045e173
sumC: 3.140148927829016e176

temp: 10   Cv: -1.0330175795496363e64
Zest: 1.0825458136851048e55
term1: 0.0020681599200000006
sumA: -1.0113957710145546e-22
sumB: -1.0213781876044355e86
sumC: 2.2335859406648293e88

temp: 15   Cv: -1.1913712281299957e42
Zest: 4.3424733951566804e36
term1: 0.0020681599200000006
sumA: -1.805130858619772e-15
sumB: -6.599916135946702e56
sumC: 6.66745673841453e58

temp: 20   Cv: -1.0919067644178275e31
Zest: 2.750316074322079e27
term1: 0.0020681599200000006
sumA: -6.4713328424424765e-12
sumB: -1.687298105355546e42
sumC: 9.787341149773076e43

temp: 25   Cv: -2.3823102535653678e24
Zest: 8.324724533123618e21
term1: 0.0020681599200000006
sumA: -7.98919169598466e-10
sumB: -2.981916499482894e33
sumC: 1.121385723847705e35

temp: 30   Cv: -8.188478581710006e19
Zest: 1.7419193672230863e18
term1: 0.0020681599200000006
su

sumB: -48.880739730425546
sumC: 15.570404715891774

temp: 305   Cv: 0.184137857594747
Zest: 45.064162220221405
term1: 0.0020681599200000006
sumA: -0.0027059938793871086
sumB: -44.051628449794244
sumC: 13.576621411873003

temp: 310   Cv: 0.17619765608895588
Zest: 42.1352541824201
term1: 0.0020681599200000006
sumA: -0.0027094806672196513
sumB: -39.835801193299694
sumC: 11.884831145978191

temp: 315   Cv: 0.16857427738052455
Zest: 39.48084728743358
term1: 0.0020681599200000006
sumA: -0.0027116513569944867
sumB: -36.14104972871024
sumC: 10.443001691484936

temp: 320   Cv: 0.16127916291073596
Zest: 37.068960633393495
term1: 0.0020681599200000006
sumA: -0.0027125808168077577
sumB: -32.890943883282354
sumC: 9.2090547314064

temp: 325   Cv: 0.15431587456591028
Zest: 34.87198699608864
term1: 0.0020681599200000006
sumA: -0.0027123405496934654
sumB: -30.02183284395625
sumC: 8.14878650974396

temp: 330   Cv: 0.14768225617144864
Zest: 32.86601362507779
term1: 0.0020681599200000006
sumA: -0.00271099

In [ ]:
for i in 1:length(sample_energies)
    print("sample energy "*string(i)*": ",sample_energies[i],"\n")
end

plot(T, Cv, title = "Heat Capacity", ylabel = "Cv (eV/K)", xlabel = "Temp (K)",
            dpi = 150, aspectratio = 1)

In [ ]:
#=
TO DO LIST (heat capacity):
- Relate energy or velocities to temperature
- Record high energies eliminated
- Cv = -(δ/δT δlnZ/δβ)v
    - β = 1/(kT)
    - k = kB = Bolzmann constant
    - En = energy recorded at level n
    UNITLESS
    - K = num_samples
    - wn = α^n - α^(n+1)
    - α = (K-1)/K
    - N = number of particles in a sample
    - Zest  = Σn [α^n - α^(n+1)] * [exp(-β*E(xj))]
        (this only works for our specific choice of α)
- GOAL: T in kelvins, Cv in eV per kelvin
* E_tot can exceed its original value... value should be constant
=#


# HEAT CAPACITY array as function of temperature array
# Make an array of temperatures

# For each T:
# Use a loop to iterate n for every En in sample_energies
# α = (K-1)/K
# β = 1/(kB*T)
# Zest  = Σn [α^n - α^(n+1)] * [exp(-β*E(xj))]
# E(xj) represents a sample between energy levels En and En+1
        # I think that E(xj) is either just En or maybe E0
"""Figure out meaning of E(xj)"""
# Cv(T) = 3*N/2*kB - sum((α^n - α^(n+1))*En*exp(-β*En)/(kB*T^2))/Zest^2 * \
#                    sum(wn*En*exp(-β*En)) + 1/Zest*sum((α^n - α^(n+1))*En^2*exp(-β*En)/(kB*T^2))

In [30]:
?convert

search: convert code_native @code_native



```
convert(T, x)
```

Convert `x` to a value of type `T`.

If `T` is an [`Integer`](@ref) type, an [`InexactError`](@ref) will be raised if `x` is not representable by `T`, for example if `x` is not integer-valued, or is outside the range supported by `T`.

# Examples

```jldoctest
julia> convert(Int, 3.0)
3

julia> convert(Int, 3.5)
ERROR: InexactError: Int64(3.5)
Stacktrace:
[...]
```

If `T` is a [`AbstractFloat`](@ref) or [`Rational`](@ref) type, then it will return the closest value to `x` representable by `T`.

```jldoctest
julia> x = 1/3
0.3333333333333333

julia> convert(Float32, x)
0.33333334f0

julia> convert(Rational{Int32}, x)
1//3

julia> convert(Rational{Int64}, x)
6004799503160661//18014398509481984
```

If `T` is a collection type and `x` a collection, the result of `convert(T, x)` may alias all or part of `x`.

```jldoctest
julia> x = Int[1, 2, 3];

julia> y = convert(Vector{Int}, x);

julia> y === x
true
```


In [32]:
"""Same values are recorded in anim arrays for each sample"""
# They're taking the values of the third sample...

# length of x_for_anim[k] changed when I got rid of a sample....
# each sample is being recorded in each animation in sequence?
# currently 200 frames per sample...
# 50 frames recorded (T = 500, s = 10, T/s=50) for each sample over each element of for_anim
# 150 frames total for 3 samples
# done 4 times since num_loops=4

length(x_for_anim[1])


800

In [ ]:
#=
# Heat Capacity Calculation
#= from pg.1-9 of "Efficient sampling of aatomic configuration spaces"
 by Lívia B. Pártay, Albert P. Bartók, and Gábor Csányi =#

Cv = 3*N/2*kB - Σ_n(ω_n*E_n*exp(-β*E_n)/(kB*T^2))/Zest^2 * Σ_n(ω_n*E_n*exp(-β*E_n))
 + 1/Zest * Σ_n(ω_n * E_n^2 * exp(-β*E_n)/(kB*T^2))

N = number of particles
kB = Boltzmann constant
ω_n = α^n - α^(n+1)
α = (K-1)/K
K = number of samples
Zest = Σ_n(ω_n)*Σ_j(exp(-β*E(xj)))
E_n < E(xj) < E_n+1
E = potential energy #lennard jones potential!
T = temperature in K


=#

In [ ]:
"""TO DO: Figure out how to plot all samples at once"""

#=

println("This cell is almost done...")

function p(k,i)
    lim = L
    dotsize = 3

    scatter(x_for_anim[k][i], y_for_anim[k][i], markersize = dotsize, 
            xlims = (-lim,lim), ylims = (-lim,lim),
            ylabel = plotnames[k], size = (800,800), aspectratio = 1)
    
end



subplots_anim = @animate for i = 1:length(x_for_anim[1])

    plot(p(1,i), p(2,i), p(3,i), p(4,i), p(5,i), p(6,i))
    
end

# DO NOT RUN THIS UNTIL YOU KNOW WHERE IT WILL SAVE TO
# gif(subplots_anim,"all_methods.gif")

=#

println("This cell is almost done...")

function mp4_p(k,i,plotname)
    lim = L
    dotsize = 3

    scatter(x_for_anim[k][i], y_for_anim[k][i], title = plotname,
            markersize = dotsize, xlims = (-lim,lim), ylims = (-lim,lim), 
            dpi = 150, aspectratio = 1)
    
end


# This will hopefully work for the first three samples of an group
samples_anim = @animate for i = 1:length(x_for_anim[1])

    # TO DO: make a loop that will do this for any number of samples    
    plot(mp4_p(1,i,plotnames[1]),mp4_p(2,i,plotnames[2]),mp4_p(3,i,plotnames[3]),mp4_p(4,i,plotnames[4]))

end

# mp4(LJ_anim,"./tmp_animations/LennardJones.mp4");
mp4(samples_anim,"./tmp_animations/all_samples.mp4");


In [ ]:
# Creating mp4 files of each sample

"""TO DO: Create large mp4 showing all samples walking simultaneously"""

println("This cell is almost done...")

function mp4_p(k,i,plotnames)
    lim = L
    dotsize = 3

    scatter(x_for_anim[k][i], y_for_anim[k][i], title = plotnames[k],
            markersize = dotsize, xlims = (-lim,lim), ylims = (-lim,lim), 
            dpi = 150, aspectratio = 1)
    
end


for k in 1:length(animfile_names)   
    LJ_anim = @animate for i = 1:length(x_for_anim[k])

        plot(mp4_p(k,i,plotnames))

    end


    # mp4(LJ_anim,"./tmp_animations/LennardJones.mp4");
    mp4(LJ_anim,"./tmp_animations/"*animfile_names[k]*".mp4");
end

In [ ]:
println("This cell is almost done...")

function p(k,i)
    lim = L
    dotsize = 3

    scatter(x_for_anim[k][i], y_for_anim[k][i], title = plotnames[k],
            markersize = dotsize, 
            xlims = (-lim,lim), ylims = (-lim,lim), 
            size = (350,350), dpi=150, aspectratio = 1)
    
end


for k in 1:length(animfile_names)
    plot_anim = @animate for i = 1:length(x_for_anim[k])

        plot(p(k,i))

    end

    # gif(plot_anim,"./tmp_animations/MD.gif")
    gif(plot_anim,"./tmp_animations/"*animfile_names[k]*".gif")
end

In [1]:
?rand

search: rand randn transcode macroexpand @macroexpand @macroexpand1



```
rand([rng=GLOBAL_RNG], [S], [dims...])
```

Pick a random element or array of random elements from the set of values specified by `S`; `S` can be

  * an indexable collection (for example `1:9` or `('x', "y", :z)`),
  * an `AbstractDict` or `AbstractSet` object,
  * a string (considered as a collection of characters), or
  * a type: the set of values to pick from is then equivalent to `typemin(S):typemax(S)` for integers (this is not applicable to [`BigInt`](@ref)), and to $[0, 1)$ for floating point numbers;

`S` defaults to [`Float64`](@ref).

!!! compat "Julia 1.1"
    Support for `S` as a tuple requires at least Julia 1.1.


# Examples

```julia-repl
julia> rand(Int, 2)
2-element Array{Int64,1}:
 1339893410598768192
 1575814717733606317

julia> rand(MersenneTwister(0), Dict(1=>2, 3=>4))
1=>2
```

!!! note
    The complexity of `rand(rng, s::Union{AbstractDict,AbstractSet})` is linear in the length of `s`, unless an optimized method with constant complexity is available, which is the case for `Dict`, `Set` and `BitSet`. For more than a few calls, use `rand(rng, collect(s))` instead, or either `rand(rng, Dict(s))` or `rand(rng, Set(s))` as appropriate.



In [52]:
### Calculate particle paths using each method and record points for plotting

"""THIS is now entailed by the walk_record function"""

#Try running this with the same list twice, see if it pops out with the same answer twice

#=
x_for_anim = []
y_for_anim = []


for i in 1:length(samples_list)
    
    
    x = deepcopy(samples_list[i][1])
    y = deepcopy(samples_list[i][2])
    vx = deepcopy(samples_list[i][3])
    vy = deepcopy(samples_list[i][4])
    xprev = deepcopy(samples_list[i][5])
    yprev = deepcopy(samples_list[i][6])
    
    
    x_anim_tmp = []
    y_anim_tmp = []
    
    # Initialize force vectors and arrays for plotting
    Fx = zeros(K)
    Fy = zeros(K)

    
    for n in 1:T

        # Calculating net force on each particle one at a time
        comp_path(K,L,ς,sqς,x,y,Fx,Fy,oneimage)

        # Updating coordinates and velocities Verlet method
        newposition(K,L,x,xprev,Fx,dt,vx,n,T)
        newposition(K,L,y,yprev,Fy,dt,vy,n,T)

        
        # Record the points every few time steps
        if n%s == 0
            recordpositions(K,x,y,x_anim_tmp,y_anim_tmp)
            
            # Final timestep, record the velocities and new initial values
            if n == T
                #recordvelocities(K,vx,vy,vx_tmp,vy_tmp)
                
                samples_list[i][1] = deepcopy(x)
                samples_list[i][2] = deepcopy(y)
                samples_list[i][3] = deepcopy(vx)
                samples_list[i][4] = deepcopy(vy)
                samples_list[i][5] = deepcopy(xprev)
                samples_list[i][6] = deepcopy(yprev)
                
            end
            
        end

        
    end

    
    # Use velocities/positions to calculate kinetic/potential energies and find total energy
    E_total, PE_total, KE_total = get_energy(K,L,m,ς,sqς,x,y,vx,vy)
    
    print("Total potential energy: ",PE_total,"\n")
    print("  Total kinetic energy: ",KE_total,"\n")
    print("          Total energy: ",E_total,"\n")    
        
        
    
    push!(x_for_anim, deepcopy(x_anim_tmp))
    push!(y_for_anim, deepcopy(y_anim_tmp))

    
    println("Sample ", samples_list[i][7], " has finished walking.\n\n")

end

#=
Made it to the end!
Total potential energy: -72.23310480676912
  Total kinetic energy: 27.604138025714413
          Total energy: -44.62896678105471
Sample 1 has finished walking.


Made it to the end!
Total potential energy: -69.17169434856865
  Total kinetic energy: 27.23576681711393
          Total energy: -41.93592753145472
Sample 2 has finished walking.


Made it to the end!
Total potential energy: -61.57677134701303
  Total kinetic energy: 23.537051918650356
          Total energy: -38.03971942836267
Sample 3 has finished walking.
=#

Total potential energy: -13.473462963354327
  Total kinetic energy: 5.316470902009678
          Total energy: -8.15699206134465
Sample 1 has finished walking.


Total potential energy: -17.584754915687455
  Total kinetic energy: 6.604680204616985
          Total energy: -10.98007471107047
Sample 2 has finished walking.


Total potential energy: -15.90108365152345
  Total kinetic energy: 6.211512754926162
          Total energy: -9.689570896597289
Sample 3 has finished walking.




In [144]:
x_for_anim = fill([],length(samples_list))
x_for_anim[1]=[1234]
push!(x_for_anim[1],23)
print(x_for_anim)

Array{Any,1}[[1234, 23], [], []]